<a href="https://colab.research.google.com/github/LJMfintech/Financial_Data_Analysis_1/blob/main/DataAnalysisHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds
!pip install numpy_financial

In [2]:
import wrds
import numpy_financial as npf
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [3]:
conn = wrds.Connection()

Enter your WRDS username [root]:leeskku2025
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# 1번 문제

In [4]:
lib_list = conn.list_libraries()
print(type(lib_list))
print(lib_list)

<class 'list'>
['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common', 'auditsmp', 'auditsmp_all', 'bank', 'bank_all', 'bank_premium_samp', 'banksamp', 'block', 'block_all', 'boardex_trial', 'boardsmp', 'bvd_amadeus_trial', 'bvd_bvdbankf_trial', 'bvd_orbis_trial', 'bvdsamp', 'calcbench_trial', 'calcbnch', 'candid_samp', 'cboe', 'cboe_all', 'cboe_sample', 'cboesamp', 'cddsamp', 'ciq', 'ciq_capstrct', 'ciq_common', 'ciqsamp', 'ciqsamp_capstrct', 'ciqsamp_common', 'ciqsamp_keydev', 'ciqsamp_pplintel', 'ciqsamp_ratings', 'ciqsamp_transactions', 'ciqsamp_transcripts', 'cisdmsmp', 'columnar', 'comp', 'comp_execucomp', 'comp_global_daily', 'comp_na_daily_all', 'compsamp', 'compsamp_all', 'compsamp_snapshot', 'contrib', 'contrib_as_filed_financials', 'contrib_ceo_turnover', 'contrib_char_returns', 'contrib_corporate_culture', 'contrib_general', 'contrib_global_factor', 'contrib_intangible_value', 'contrib_kpss', 'contrib_liva', 'crsp', 'crsp_a_indexes', 'crsp_a_stock', 'crsp_a_t

In [5]:
conn.list_tables(library='comp')

['aco_amda',
 'aco_imda',
 'aco_indfnta',
 'aco_indfntq',
 'aco_indfntytd',
 'aco_indsta',
 'aco_indstq',
 'aco_indstytd',
 'aco_notesa',
 'aco_notesq',
 'aco_notessa',
 'aco_notesytd',
 'aco_pnfnda',
 'aco_pnfndq',
 'aco_pnfndytd',
 'aco_pnfnta',
 'aco_pnfntq',
 'aco_pnfntytd',
 'aco_transa',
 'aco_transq',
 'aco_transsa',
 'aco_transytd',
 'adsprate',
 'asec_amda',
 'asec_imda',
 'asec_notesa',
 'asec_notesq',
 'asec_transa',
 'asec_transq',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'chars',
 'co_aacctchg',
 'co_aaudit',
 'co_acthist',
 'co_adesind',
 'co_adjfact',
 'co_afnd1',
 'co_afnd2',
 'co_afnddc1',
 'co_afnddc2',
 'co_afntind1',
 'co_afntind2',
 'co_ainvval',
 'co_amkt',
 'co_busdescl',
 'co_cotype',
 'co_f

In [6]:
country = conn.get_table(library='comp', table='g_secm', obs = 5)
country

,gvkey,iid,datadate,conm,ajexm,ajpm,isalrt,cshtrm,curcdm,prccm,...,dvpspm_fn,dvpsxm_fn,epf,exchg,isin,secstat,sedol,tpci,fic,loc
0,000005,01W,2006-01-31,<NA>,1.0,1.0,<NA>,<NA>,USD,10864.86,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,000005,01W,2006-02-28,<NA>,1.0,1.0,<NA>,<NA>,USD,10993.41,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,000005,01W,2006-03-31,<NA>,1.0,1.0,<NA>,<NA>,USD,11109.32,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,000005,01W,2006-04-30,<NA>,1.0,1.0,<NA>,<NA>,USD,11367.14,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,000005,01W,2006-05-31,<NA>,1.0,1.0,<NA>,<NA>,USD,11168.31,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [17]:
# a: 월별 기업 데이터
# b: 기업명 및 국가
# 기업(gvkey) 기준으로 매칭

data_set = conn.raw_sql("""select gvkey, iid, datadate, conm, ajexm, ajpm, cshtrm, curcdm, prccm, prchm, prclm, dvpspm, dvpsxm, fic

                          from comp.g_secm
                          where fic IN('GBR', 'DEU', 'JPN', 'FRA', 'AUS', 'CHN', 'IND', 'BRA', 'ZAF', 'TUR')
                          and datadate between '2020-03-01' and '2024-12-31'
                          """)
print(data_set.shape)



(1546523, 14)


In [10]:
data_set.head()

,gvkey,iid,datadate,ajexm,ajpm,curcdm,prccm,prchm,prclm,curcddvm,dvpspm,dvpsxm,conm,fic
0,001932,01W,2020-03-31,1.0,1.0,GBP,27.59,32.445,23.82,GBP,0.0,0.526,BRITISH AMER TOBACCO PLC,GBR
1,001932,02W,2020-03-31,1.0,1.0,ZAR,598.66,645.17,498.0,ZAR,0.0,10.789,BRITISH AMER TOBACCO PLC,GBR
2,001932,03W,2020-03-31,1.0,1.0,EUR,30.98,37.265,25.65,EUR,0.0,0.579,BRITISH AMER TOBACCO PLC,GBR
3,001932,04W,2020-03-31,1.0,1.0,GBP,27.385,32.53,23.685,GBP,0.0,0.526,BRITISH AMER TOBACCO PLC,GBR
4,001932,07W,2020-03-31,1.0,1.0,CHF,34.44,34.44,34.44,CHF,0.0,0.616,BRITISH AMER TOBACCO PLC,GBR


In [11]:
data_set.to_csv('data_set.csv')

# 2번 문제

In [24]:
data_set = data_set.sort_values(['gvkey', 'datadate'])

# 2. 월별 수익률 계산 (조정종가 + 배당)
data_set['prccm_adj'] = data_set['prccm'] * data_set['ajexm']
data_set['prccm_adj_prev'] = data_set.groupby('gvkey')['prccm_adj'].shift(1)
data_set['div'] = data_set['dvpsxm'].fillna(0) if 'dvpsxm' in data_set.columns else 0
data_set['ret'] = (data_set['prccm_adj'] + data_set['div']) / data_set['prccm_adj_prev'] - 1

# 3. 시가총액 계산 (주가 × 발행주식수, 예: shares_outstanding 변수가 cshtrm, csho 등으로 제공된다 가정)
if 'cshtrm' in data_set.columns:
    data_set['mktcap'] = data_set['prccm'] * data_set['cshtrm']  # cshtrm NA시, quarterly/annual 데이터를 병합해서 채워넣기 가능
else:
    data_set['mktcap'] = np.nan # 결측 우선 처리, 이후 필요변수 병합해야 함

# 4. 국가-월별 동등가중(EW) 수익률
ew_ret = data_set.groupby(['fic', 'datadate'])['ret'].mean().reset_index(name='ew_ret')

# 5. 국가-월별 시가총액가중(VW) 수익률
def vw_avg(x):
    valid = (x['mktcap'] > 0) & (x['ret'].notnull())
    return np.average(x.loc[valid, 'ret'], weights=x.loc[valid, 'mktcap']) if valid.any() else np.nan

vw_ret = data_set.groupby(['fic', 'datadate']).apply(vw_avg).reset_index(name='vw_ret')

# 6. 결과 출력/확인
print("기업별 월별 수익률 예시:")
# print(data_set[['gvkey', 'conm', 'datadate', 'ret']].head())
firm_monthly_ret = data_set.groupby(['gvkey', 'conm', 'datadate'])['ret'].mean().reset_index()
print(firm_monthly_ret.head())

print("기업별 시가총액 예시:")
# print(data_set[['gvkey', 'conm', 'datadate', 'mktcap']].head())
firm_monthly_mktcap = data_set.groupby(['gvkey', 'conm', 'datadate'])['mktcap'].sum().reset_index()
print(firm_monthly_mktcap.head())

print("\n국가별 EW 월별 수익률 예시:")
print(ew_ret.head())

print("\n국가별 VW 월별 수익률 예시:")
print(vw_ret.head())

/tmp/ipython-input-2159093724.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vw_ret = data_set.groupby(['fic', 'datadate']).apply(vw_avg).reset_index(name='vw_ret')


기업별 월별 수익률 예시:
     gvkey                      conm    datadate       ret
0   001932  BRITISH AMER TOBACCO PLC  2020-03-31  5.080814
1   001932  BRITISH AMER TOBACCO PLC  2020-04-30  4.185359
2   001932  BRITISH AMER TOBACCO PLC  2020-05-31   3.93544
3   001932  BRITISH AMER TOBACCO PLC  2020-06-30  3.906054
4   001932  BRITISH AMER TOBACCO PLC  2020-07-31  4.271441
5   001932  BRITISH AMER TOBACCO PLC  2020-08-31  4.121512
6   001932  BRITISH AMER TOBACCO PLC  2020-09-30  3.940944
7   001932  BRITISH AMER TOBACCO PLC  2020-10-31  3.807989
8   001932  BRITISH AMER TOBACCO PLC  2020-11-30  3.728691
9   001932  BRITISH AMER TOBACCO PLC  2020-12-31   7.29061
10  001932  BRITISH AMER TOBACCO PLC  2021-01-31  7.443317
11  001932  BRITISH AMER TOBACCO PLC  2021-02-28  7.819773
12  001932  BRITISH AMER TOBACCO PLC  2021-03-31  7.342994
13  001932  BRITISH AMER TOBACCO PLC  2021-04-30  7.295424
14  001932  BRITISH AMER TOBACCO PLC  2021-05-31  7.130209
15  001932  BRITISH AMER TOBACCO PLC  202

# 3번 문제

In [27]:
# datadate 컬럼을 datetime 타입으로 변환 (중요!)
data_set['datadate'] = pd.to_datetime(data_set['datadate'])

# 새 컬럼 'period' 생성
def classify_period(date):
    if pd.Timestamp('2020-03-01') <= date <= pd.Timestamp('2021-12-31'):
        return 'COVID-19 Crisis Period'
    elif pd.Timestamp('2022-01-01') <= date <= pd.Timestamp('2024-12-31'):
        return 'Post-crisis Recovery Period'
    else:
        return 'Other'

data_set['period'] = data_set['datadate'].apply(classify_period)

# 결과 예시
print(data_set[['datadate', 'period']].drop_duplicates().head(15))

# 필요시 각 구간별 데이터 분리도 가능
covid_period    = data_set[data_set['period'] == 'COVID-19 Crisis Period']
recovery_period = data_set[data_set['period'] == 'Post-crisis Recovery Period']

         datadate                  period
0      2020-03-31  COVID-19 Crisis Period
23652  2020-04-30  COVID-19 Crisis Period
47295  2020-05-31  COVID-19 Crisis Period
71000  2020-06-30  COVID-19 Crisis Period
94787  2020-07-31  COVID-19 Crisis Period
118686 2020-08-31  COVID-19 Crisis Period
142676 2020-09-30  COVID-19 Crisis Period
166776 2020-10-31  COVID-19 Crisis Period
190974 2020-11-30  COVID-19 Crisis Period
215236 2020-12-31  COVID-19 Crisis Period
239754 2021-01-31  COVID-19 Crisis Period
264362 2021-02-28  COVID-19 Crisis Period
289084 2021-03-31  COVID-19 Crisis Period
314124 2021-04-30  COVID-19 Crisis Period
339265 2021-05-31  COVID-19 Crisis Period


# 4번 문제

In [36]:
pd.set_option("display.precision", 3)
data_set.describe()



,datadate,ajexm,ajpm,cshtrm,prccm,prchm,prclm,dvpspm,dvpsxm,prccm_adj,prccm_adj_prev,div,ret,mktcap
count,1546523,1546523.0,1546523.0,1361016.0,1497296.0,1497296.0,1497296.0,1546523.0,1546523.0,1497296.0,1475377.0,1546523.0,1471804.0,1361016.0
mean,2022-09-15 09:41:31.337535488,1.421,1.258,149779.619,1151.339,1193.911,1099.002,2.31,2.37,9229.478,8905.725,2.37,<NA>,5391456.0
min,2020-03-31 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,0.0
25%,2021-06-30 00:00:00,1.0,1.0,320.475,3.71,4.0,3.45,0.0,0.0,3.71,3.71,0.0,<NA>,4342.069
50%,2022-09-30 00:00:00,1.0,1.0,6928.533,14.63,15.86,13.52,0.0,0.0,15.26,15.2,0.0,<NA>,362550.379
75%,2023-11-30 00:00:00,1.0,1.0,98387.873,98.19,105.8,90.45,0.0,0.0,121.0,119.36,0.0,<NA>,2193899.184
max,2024-12-31 00:00:00,3000.0,3000.0,112043152.6,260111111.111,260111111.111,260000000.0,15384.0,15384.0,567127500.0,567127500.0,15384.0,<NA>,8953707840.0
std,NaN,18.69,18.489,568208.02,213374.311,213413.046,213201.846,107.677,108.773,1567211.391,1530182.178,108.773,<NA>,47039368.303


In [42]:
# 10개국 리스트
country_list = ['GBR', 'DEU', 'JPN', 'FRA', 'AUS', 'CHN', 'IND', 'BRA', 'ZAF', 'TUR']


# 10개국 데이터 필터링
data_10 = data_set[data_set['fic'].isin(country_list)]

# EW 수익률 계산
ew_ret = data_10.groupby(['fic', 'datadate'])['ret'].mean().reset_index(name='ew_ret')
# VW 수익률 계산 함수
def vw_avg(x):
    cond = (x['mktcap'] > 0) & x['ret'].notnull()
    if cond.any():
        return np.average(x.loc[cond,'ret'], weights=x.loc[cond,'mktcap'])
    else:
        return np.nan
vw_ret = data_10.groupby(['fic', 'datadate']).apply(vw_avg).reset_index(name='vw_ret')

# 연월(Year-Month) 컬럼 추가 (datadate가 datetime 아니면 변환 필요)
data_10['datadate'] = pd.to_datetime(data_10['datadate'])
ew_ret['year_month'] = ew_ret['datadate'].dt.to_period('M')
vw_ret['year_month'] = vw_ret['datadate'].dt.to_period('M')

# 피벗(pivot)해서 국가별 row, 월별 수익률이 열이 되도록 변환
ew_pivot = ew_ret.pivot(index='fic', columns='year_month', values='ew_ret').reset_index()
vw_pivot = vw_ret.pivot(index='fic', columns='year_month', values='vw_ret').reset_index()

print("EW 수익률 (국가별 1행, 월별 열):")
print(ew_pivot.head())

print("\nVW 수익률 (국가별 1행, 월별 열):")
print(vw_pivot.head())

KeyError: 'Column not found: ret'